In [8]:
import numpy as np
import numpy.ma as ma
import pandas as pd

from utils.predictorjpg import *

The purpose of this notebook is to emsembling existing kaggle submission files by simple averaging and then apply thresholding to optimize F2 score.  

The highest LB (F2) score for a single model (resnet50) is 0.92912.  With ensembling, we get 0.93146 which is an improvement of 0.00897.

Due to lack of time to rerun training, the thresholding is approximate by averaging across thresholds for each model.

In [9]:
data_dir = 'D:/Downloads/amazon/'
sample_submission_filepath = data_dir + 'sample_submission_v2.csv'
real_submission_filepath = data_dir + 'my_submissions/ensemble/mean_ensemble_submission.csv'

labels = ['slash_burn', 'clear', 'blooming', 'primary', 'cloudy', 'conventional_mine', 'water', 'haze', 'cultivation', 'partly_cloudy', 'artisinal_mine', 'habitation', 'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']

In [10]:
submit_dir = data_dir + 'my_submissions/'

vgg19 = np.load(submit_dir + 'tta_submission_vgg19_20170719-113132.csv.h5.npy')
resnet50 = np.load(submit_dir + 'tta_submission_resnet50_20170718-085651.csv.h5.npy')
vgg16 = np.load(submit_dir + 'tta_submission_vgg16_20170720-065517.csv.h5.npy')
#vgg16 = np.load(submit_dir + 'tta_submission_vgg16_20170717-171015.csv.h5.npy')
densenet121 = np.load(submit_dir + 'tta_submission_densenet121_20170717-161234.csv.h5.npy')

aggregation = []
aggregation.append(vgg19)
aggregation.append(resnet50)
aggregation.append(vgg16)
aggregation.append(densenet121)

avg = np.mean(aggregation, axis=0)
print(avg.shape)

(61191L, 17L)


In [11]:
# these threshold number are determined by experiments carried out after individual model training.
a = [0.29, 0.08, 0.31, 0.17, 0.04, 0.25, 0.14, 0.17, 0.21, 0.11, 0.14, 0.17, 0.15, 0.21, 0.20, 0.18, 0.12]
b = [0.26, 0.12, 0.26, 0.21, 0.05, 0.20, 0.24, 0.29, 0.20, 0.05, 0.04, 0.14, 0.18, 0.12, 0.19, 0.13, 0.25]
c = [0.12, 0.07, 0.15, 0.32, 0.12, 0.16, 0.13, 0.36, 0.27, 0.11, 0.14, 0.15, 0.17, 0.06, 0.16, 0.16, 0.46]
d = [0.64, 0.09, 0.30, 0.20, 0.25, 0.48, 0.22, 0.45, 0.25, 0.11, 0.06, 0.23, 0.28, 0.19, 0.24, 0.23, 0.42]

agg_threshold = []
agg_threshold.append(a)
agg_threshold.append(b)
agg_threshold.append(c)
agg_threshold.append(d)
avg_threshold = np.mean(agg_threshold, axis=0)
print(avg_threshold)

[ 0.3275  0.09    0.255   0.225   0.115   0.2725  0.1825  0.3175  0.2325
  0.095   0.095   0.1725  0.195   0.145   0.1975  0.175   0.3125]


In [12]:
binary_predictions = (np.array(avg) > avg_threshold).astype(int)
predict_df = pd.DataFrame(binary_predictions, columns = labels)

df_submission = pd.read_csv(sample_submission_filepath)
submit_df = submission_dataframe(df_submission, predict_df)
submit_df.to_csv(real_submission_filepath, index=False)

100%|███████████████████████████████████████████████████████████████████████████| 61191/61191 [01:03<00:00, 962.23it/s]
